In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import spacy
from spacy.training import Example
import random
import pandas as pd
from nltk.corpus import stopwords
import nltk
from sklearn.model_selection import train_test_split

nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

Mounted at /content/drive


#Sentiment Analysis

##High - High Quality

This data set is saved in csv file which contains 400 dataset that has sentiments from customers about the ice cream taste in a ice cream store, the datase is collected with the help of kaggle and have an original size of almost 5000+ dataset.

In [ ]:

# Load a blank model and add text classifier
# Path to the CSV file in Google Drive
csv_file_path5 = '/content/drive/MyDrive/4th Year/ELECTIVE 4/Exercise 4.1/Sentiment Analysis/High_size_HIGH_quality_Sentiment.csv'

# Load the CSV file using pandas
df = pd.read_csv(csv_file_path5)

# Function to remove stopwords
def remove_stopwords(text):
    words = text.split()
    filtered_words = [word for word in words if word.lower() not in stop_words]
    return ' '.join(filtered_words)

# Convert the DataFrame to the format required by spaCy
data = []
for index, row in df.iterrows():
    text = remove_stopwords(row['text'])  # Remove stopwords
    label = {'Satisfied': row['Final_Rating'] == 'Satisfied', 'Not Satisfied': row['Final_Rating'] == 'Not Satisfied'}
    data.append((text, {'cats': label}))

# Split the data into training and testing sets (80% train, 20% test)
train_data5, test_data5 = train_test_split(data, test_size=0.3, random_state=42)

# Load a blank spaCy model
nlp = spacy.blank("en")

# Add the text classification pipeline
textcat = nlp.add_pipe("textcat")

# Add labels for the text classification
textcat.add_label("Satisfied")
textcat.add_label("Not Satisfied")

# Training the model
def train_model(data, n_iter=10):
    random.shuffle(data)
    optimizer = nlp.begin_training()
    for epoch in range(n_iter):
        losses = {}
        for text, annotations in data:
            doc = nlp.make_doc(text)
            example = Example.from_dict(doc, annotations)
            nlp.update([example], drop=0.5, losses=losses)
        print(f"Epoch {epoch + 1}/{n_iter} Losses: {losses}")

train_model(train_data5)

# Function to predict sentiment
def predict_sentiment(text):
    text = remove_stopwords(text)  # Remove stopwords from user input
    doc = nlp(text)
    return doc.cats

# Evaluate the model on the test data
y_true = []
y_pred = []

for text, annotations in test_data5:
    doc = nlp(text)
    y_true.append(max(annotations["cats"], key=annotations["cats"].get))
    y_pred.append(max(doc.cats, key=doc.cats.get))

# Print evaluation results
from sklearn.metrics import classification_report

report = classification_report(y_true, y_pred, target_names=["Not Satisfied", "Satisfied"])
print("Performance Evaluation Report:\n", report)

# User input and sentiment analysis
"""
while True:
    user_input = input("Enter a text for sentiment analysis (or type 'exit' to quit): ")
    if user_input.lower() == 'exit':
        break
    prediction = predict_sentiment(user_input)
    print("Sentiment Prediction:")
    print(prediction)
"""


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Epoch 1/10 Losses: {'textcat': 73.2360592186451}
Epoch 2/10 Losses: {'textcat': 66.51440756539341}
Epoch 3/10 Losses: {'textcat': 59.37238605113987}
Epoch 4/10 Losses: {'textcat': 47.755903024564425}
Epoch 5/10 Losses: {'textcat': 41.60347930388314}
Epoch 6/10 Losses: {'textcat': 34.069622301405616}
Epoch 7/10 Losses: {'textcat': 20.107663974529416}
Epoch 8/10 Losses: {'textcat': 15.733907981540499}
Epoch 9/10 Losses: {'textcat': 9.756341760908066}
Epoch 10/10 Losses: {'textcat': 8.754340456397685}
Performance Evaluation Report:
                precision    recall  f1-score   support

Not Satisfied       0.89      0.63      0.74        62
    Satisfied       0.70      0.91      0.79        58

     accuracy                           0.77       120
    macro avg       0.79      0.77      0.76       120
 weighted avg       0.80      0.77      0.76       120



'\nwhile True:\n    user_input = input("Enter a text for sentiment analysis (or type \'exit\' to quit): ")\n    if user_input.lower() == \'exit\':\n        break\n    prediction = predict_sentiment(user_input)\n    print("Sentiment Prediction:")\n    print(prediction)\n'

# POS Tagging


The dataset used has 50 data which contains text and it was predfined according to its part of speeach. Each text in are defined whether it is a 'noun', 'verb', 'adp', etc.

##High - High Quality

In [ ]:
import spacy
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
from sklearn.model_selection import train_test_split

nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
# Load pre-trained spaCy model
nlp = spacy.load("en_core_web_sm")

# Sample data with individual words and their corresponding labels
data5 = [("Artificial Intelligence is transforming industries across the globe.",["ADJ", "NOUN", "AUX", "VERB", "NOUN", "ADP", "DET", "NOUN"]),
    ("The quick brown fox jumps over the lazy dog.",["DET", "ADJ", "ADJ", "NOUN", "VERB", "ADP", "DET", "ADJ", "NOUN"]),
    ("Data scientists analyze vast amounts of information every day.",["NOUN", "NOUN", "VERB", "ADJ", "NOUN", "ADP", "NOUN", "DET", "NOUN"]),
    ("Climate change is a pressing global issue that requires immediate action.",["NOUN", "NOUN", "AUX", "DET", "ADJ", "ADJ", "NOUN", "PRON", "VERB", "ADJ", "NOUN"]),
    ("Renewable energy sources are crucial for a sustainable future.",["ADJ", "NOUN", "NOUN", "AUX", "ADJ", "ADP", "DET", "ADJ", "NOUN"]),
    ("Machine learning techniques improve decision making in various fields.",["NOUN", "NOUN", "NOUN", "VERB", "NOUN", "ADP", "ADJ", "NOUN"]),
    ("Natural language processing enables computers to understand human language.",["ADJ", "NOUN", "NOUN", "VERB", "NOUN", "ADP", "NOUN", "NOUN"]),
    ("The stock market is volatile and requires careful analysis.",["DET", "NOUN", "AUX", "ADJ", "CONJ", "VERB", "ADJ", "NOUN"]),
    ("Artificial Intelligence can enhance customer experience.",["ADJ", "NOUN", "AUX", "VERB", "NOUN", "NOUN"]),
    ("Robotics is the future of automation.",["NOUN", "AUX", "DET", "NOUN", "ADP", "NOUN"]),
    ("Blockchain technology revolutionizes digital transactions.", ["NOUN", "NOUN", "VERB", "ADJ", "NOUN"]),
    ("Healthcare innovations improve patient outcomes significantly.", ["NOUN", "NOUN", "VERB", "NOUN", "ADJ"]),
    ("Social media platforms connect people around the world.", ["ADJ", "NOUN", "VERB", "NOUN", "ADP", "DET", "NOUN"]),
    ("Cybersecurity measures protect sensitive information effectively.", ["NOUN", "NOUN", "VERB", "ADJ", "NOUN", "ADJ"]),
    ("Augmented reality enhances user experiences in gaming.", ["ADJ", "NOUN", "VERB", "NOUN", "ADP", "NOUN"]),
    ("Sustainable practices reduce environmental impact dramatically.", ["ADJ", "NOUN", "VERB", "ADJ", "NOUN", "ADJ"]),
    ("E-commerce continues to grow rapidly across industries.", ["NOUN", "VERB", "PART", "ADJ", "ADP", "NOUN"]),
    ("Cloud computing enables flexible data storage solutions.", ["NOUN", "NOUN", "VERB", "ADJ", "NOUN", "NOUN"]),
    ("5G technology facilitates faster internet connections.", ["NOUN", "NOUN", "VERB", "ADJ", "NOUN"]),
    ("Remote work offers new opportunities for collaboration.", ["ADJ", "NOUN", "VERB", "ADJ", "NOUN", "ADP", "NOUN"]),
    ("Artificial Intelligence will reshape the future of work.", ["ADJ", "NOUN", "AUX", "VERB", "DET", "NOUN", "ADP", "NOUN"]),
    ("The rise of renewable energy is crucial for sustainability.", ["DET", "NOUN", "ADP", "ADJ", "NOUN", "AUX", "ADJ", "ADP", "NOUN"]),
    ("Advancements in technology drive innovation and growth.", ["NOUN", "ADP", "NOUN", "VERB", "NOUN", "CONJ", "NOUN"]),
    ("Smart cities utilize technology to improve quality of life.", ["ADJ", "NOUN", "VERB", "NOUN", "ADP", "VERB", "NOUN", "ADP", "NOUN"]),
    ("Virtual reality provides immersive experiences.", ["ADJ", "NOUN", "VERB", "ADJ", "NOUN"]),
    ("The healthcare industry is evolving rapidly.", ["DET", "NOUN", "NOUN", "AUX", "VERB", "ADJ"]),
    ("Internet of Things connects everyday devices.", ["NOUN", "ADP", "NOUN", "VERB", "ADJ", "NOUN"]),
    ("Data privacy is becoming a significant concern.", ["NOUN", "NOUN", "AUX", "VERB", "DET", "ADJ", "NOUN"]),
    ("Telemedicine is transforming patient care.", ["NOUN", "AUX", "VERB", "NOUN", "NOUN"]),
    ("Blockchain offers a secure method for transactions.", ["NOUN", "VERB", "DET", "ADJ", "NOUN", "ADP", "NOUN"]),
    ("Climate action is essential for future generations.", ["NOUN", "NOUN", "AUX", "ADJ", "ADP", "ADJ", "NOUN"]),
    ("Social entrepreneurship creates positive change.", ["ADJ", "NOUN", "VERB", "ADJ", "NOUN"]),
    ("Digital marketing strategies engage customers effectively.", ["ADJ", "NOUN", "NOUN", "VERB", "NOUN", "ADJ"]),
    ("Artificial intelligence enhances decision-making processes.", ["ADJ", "NOUN", "VERB", "ADJ", "NOUN"]),
    ("Big data analytics reveals insights about consumer behavior.", ["ADJ", "NOUN", "NOUN", "VERB", "NOUN", "ADP", "NOUN"]),
    ("Sustainable agriculture practices benefit the environment.", ["ADJ", "NOUN", "NOUN", "VERB", "DET", "NOUN"]),
    ("The gig economy provides flexible work opportunities.", ["DET", "NOUN", "VERB", "ADJ", "NOUN", "NOUN"]),
    ("Smartphones have changed the way we communicate.", ["NOUN", "AUX", "VERB", "DET", "NOUN", "PRON", "VERB"]),
    ("Renewable resources are vital for energy security.", ["ADJ", "NOUN", "AUX", "ADJ", "ADP", "NOUN"]),
    ("Urbanization leads to increased infrastructure demands.", ["NOUN", "VERB", "ADP", "ADJ", "NOUN"]),
    ("Crowdsourcing taps into collective intelligence.", ["NOUN", "VERB", "ADP", "ADJ", "NOUN"]),
    ("The sharing economy encourages collaborative consumption.", ["DET", "NOUN", "VERB", "ADJ", "NOUN"]),
    ("Smart home technology automates daily tasks.", ["ADJ", "NOUN", "VERB", "ADJ", "NOUN"]),
    ("E-learning platforms are revolutionizing education.", ["NOUN", "NOUN", "AUX", "VERB", "NOUN"]),
    ("3D printing enables rapid prototyping.", ["NOUN", "VERB", "ADJ", "NOUN"]),
    ("Artificial intelligence is improving healthcare diagnostics.", ["ADJ", "NOUN", "AUX", "VERB", "NOUN", "NOUN"]),
    ("Fintech solutions streamline financial processes.", ["NOUN", "NOUN", "VERB", "ADJ", "NOUN"]),
    ("Workplace diversity enhances creativity and innovation.", ["NOUN", "NOUN", "VERB", "ADJ", "CONJ", "NOUN"]),
    ("Online communities foster collaboration and support.", ["ADJ", "NOUN", "VERB", "NOUN", "CONJ", "NOUN"]),
    ("Artificial intelligence applications are expanding rapidly.", ["ADJ", "NOUN", "NOUN", "AUX", "VERB", "ADJ"]),
    ("Remote learning offers new educational opportunities.", ["ADJ", "NOUN", "VERB", "ADJ", "ADJ", "NOUN"]),
    ("The future of work is hybrid and flexible.", ["DET", "NOUN", "ADP", "NOUN", "AUX", "ADJ"])
]

import re

all_true_pos_tags5 = []
all_predicted_pos_tags5 = []

# Iterate through training data
for text, true_pos_tags5 in data5:
    # Convert text to lowercase
    text = text.lower()

    # Remove special characters
    text = re.sub(r'[^a-z\s]', '', text)

    # Process the cleaned text with spaCy
    doc = nlp(text)

    # Extract predicted POS tags and true tags, excluding stopwords
    predicted_pos_tags5 = [token.pos_ for token in doc if not token.is_stop]
    true_pos_tags_filtered = [true_tag for token, true_tag in zip(doc, true_pos_tags5) if not token.is_stop]

    # Extend lists with true and predicted tags for evaluation
    all_true_pos_tags5.extend(true_pos_tags_filtered)
    all_predicted_pos_tags5.extend(predicted_pos_tags5)

# Ensure both lists are the same length to avoid ValueError
if len(all_true_pos_tags5) != len(all_predicted_pos_tags5):
    min_length = min(len(all_true_pos_tags5), len(all_predicted_pos_tags5))
    all_true_pos_tags5 = all_true_pos_tags5[:min_length]
    all_predicted_pos_tags5 = all_predicted_pos_tags5[:min_length]

# Evaluate
accuracy5 = accuracy_score(all_true_pos_tags5, all_predicted_pos_tags5)
precision5 = precision_score(all_true_pos_tags5, all_predicted_pos_tags5, average='weighted', zero_division=0)
recall5 = recall_score(all_true_pos_tags5, all_predicted_pos_tags5, average='weighted', zero_division=0)
f15 = f1_score(all_true_pos_tags5, all_predicted_pos_tags5, average='weighted', zero_division=0)

print("Accuracy:", accuracy5)
print("Precision:", precision5)
print("Recall:", recall5)
print("F1-Score:", f15)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Accuracy: 0.4318181818181818
Precision: 0.44836075939362324
Recall: 0.4318181818181818
F1-Score: 0.4390476389929265


#Text Categorizer

The data set used is manually inputted to the 'train data' and 'test data' object and consists of a total of 100 data. These data are phrases and categorized either it is HAM or SPAM messages.

## High - High Quality

In [ ]:
import spacy
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
from sklearn.model_selection import train_test_split
import re
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
# Load a blank model and add text classifier
nlpTC5 = spacy.blank("en")
textcat5 = nlpTC5.add_pipe("textcat", last=True)

# Add labels for classification
textcat5.add_label("SPAM")
textcat5.add_label("HAM")

# Function to lowercase and remove special characters
def process_text(text):
    return re.sub(r'[^a-z0-9\s]', '', text.lower())

# Example training data (90 samples)
train_data5 = [
    ("youve won a 1000 gift card", {"cats": {"SPAM": 1, "HAM": 0}}),
    ("hi how have you been", {"cats": {"SPAM": 0, "HAM": 1}}),
    ("claim your free prize today", {"cats": {"SPAM": 1, "HAM": 0}}),
    ("your order has shipped", {"cats": {"SPAM": 0, "HAM": 1}}),
    ("congratulations you are selected for a special offer", {"cats": {"SPAM": 1, "HAM": 0}}),
    ("lets meet for lunch soon", {"cats": {"SPAM": 0, "HAM": 1}}),
    ("urgent update your account information now", {"cats": {"SPAM": 1, "HAM": 0}}),
    ("looking forward to our meeting", {"cats": {"SPAM": 0, "HAM": 1}}),
    ("you have won a free vacation", {"cats": {"SPAM": 1, "HAM": 0}}),
    ("just checking in hope all is well", {"cats": {"SPAM": 0, "HAM": 1}}),
    ("get paid for your opinion", {"cats": {"SPAM": 1, "HAM": 0}}),
    ("dont forget about our appointment", {"cats": {"SPAM": 0, "HAM": 1}}),
    ("you are a lucky winner", {"cats": {"SPAM": 1, "HAM": 0}}),
    ("meeting is confirmed for tomorrow", {"cats": {"SPAM": 0, "HAM": 1}}),
    ("win a new car enter now", {"cats": {"SPAM": 1, "HAM": 0}}),
    ("can we reschedule our meeting", {"cats": {"SPAM": 0, "HAM": 1}}),
    ("you have a package waiting", {"cats": {"SPAM": 1, "HAM": 0}}),
    ("your subscription is about to expire", {"cats": {"SPAM": 0, "HAM": 1}}),
    ("congratulations youve been selected", {"cats": {"SPAM": 1, "HAM": 0}}),
    ("hope to see you at the event", {"cats": {"SPAM": 0, "HAM": 1}}),
    ("earn money from home", {"cats": {"SPAM": 1, "HAM": 0}}),
    ("lets catch up soon", {"cats": {"SPAM": 0, "HAM": 1}}),
    ("you have a new message", {"cats": {"SPAM": 1, "HAM": 0}}),
    ("reminder your appointment is tomorrow", {"cats": {"SPAM": 0, "HAM": 1}}),
    ("claim your free gift now", {"cats": {"SPAM": 1, "HAM": 0}}),
    ("looking forward to our lunch", {"cats": {"SPAM": 0, "HAM": 1}}),
    ("youre a winner", {"cats": {"SPAM": 1, "HAM": 0}}),
    ("dont miss this opportunity", {"cats": {"SPAM": 1, "HAM": 0}}),
    ("thank you for your purchase", {"cats": {"SPAM": 0, "HAM": 1}}),
    ("you have won a free trial", {"cats": {"SPAM": 1, "HAM": 0}}),
    ("it was great seeing you", {"cats": {"SPAM": 0, "HAM": 1}}),
    ("exclusive deal just for you", {"cats": {"SPAM": 1, "HAM": 0}}),
    ("how can i assist you today", {"cats": {"SPAM": 0, "HAM": 1}}),
    ("youve been selected for a free vacation", {"cats": {"SPAM": 1, "HAM": 0}}),
    ("your feedback is important to us", {"cats": {"SPAM": 0, "HAM": 1}}),
    ("hurry offer expires soon", {"cats": {"SPAM": 1, "HAM": 0}}),
    ("lets discuss your project", {"cats": {"SPAM": 0, "HAM": 1}}),
    ("youve won a 500 shopping spree", {"cats": {"SPAM": 1, "HAM": 0}}),
    ("hows everything going", {"cats": {"SPAM": 0, "HAM": 1}}),
    ("get a free trial of our service", {"cats": {"SPAM": 1, "HAM": 0}}),
    ("were excited to have you", {"cats": {"SPAM": 0, "HAM": 1}}),
    ("last chance to claim your prize", {"cats": {"SPAM": 1, "HAM": 0}}),
    ("thanks for being a valued customer", {"cats": {"SPAM": 0, "HAM": 1}}),
    ("youve been selected for a special offer", {"cats": {"SPAM": 1, "HAM": 0}}),
    ("dont forget to register", {"cats": {"SPAM": 0, "HAM": 1}}),
    ("congratulations you have won a gift card", {"cats": {"SPAM": 1, "HAM": 0}}),
    ("your invoice is attached", {"cats": {"SPAM": 0, "HAM": 1}}),
    ("get a new laptop for free", {"cats": {"SPAM": 1, "HAM": 0}}),
    ("hope to hear from you soon", {"cats": {"SPAM": 0, "HAM": 1}}),
    ("youve won a luxury cruise", {"cats": {"SPAM": 1, "HAM": 0}}),
    ("lets plan a call", {"cats": {"SPAM": 0, "HAM": 1}}),
    ("you have an important message", {"cats": {"SPAM": 1, "HAM": 0}}),
    ("meeting has been postponed", {"cats": {"SPAM": 0, "HAM": 1}}),
    ("claim your bonus now", {"cats": {"SPAM": 1, "HAM": 0}}),
    ("your application has been approved", {"cats": {"SPAM": 0, "HAM": 1}}),
    ("congratulations on your achievement", {"cats": {"SPAM": 1, "HAM": 0}}),
    ("lets schedule a followup", {"cats": {"SPAM": 0, "HAM": 1}}),
    ("you are eligible for a discount", {"cats": {"SPAM": 1, "HAM": 0}}),
    ("your subscription has been renewed", {"cats": {"SPAM": 0, "HAM": 1}}),
    ("win a free iphone", {"cats": {"SPAM": 1, "HAM": 0}}),
    ("it was great catching up", {"cats": {"SPAM": 0, "HAM": 1}}),
    ("youve been chosen for a special reward", {"cats": {"SPAM": 1, "HAM": 0}}),
    ("we value your opinion", {"cats": {"SPAM": 0, "HAM": 1}}),
    ("last chance to win", {"cats": {"SPAM": 1, "HAM": 0}}),
    ("dont miss this opportunity to save", {"cats": {"SPAM": 1, "HAM": 0}}),
    ("your feedback is appreciated", {"cats": {"SPAM": 0, "HAM": 1}}),
    ("you have a meeting scheduled", {"cats": {"SPAM": 0, "HAM": 1}}),
    ("youve won a new tv", {"cats": {"SPAM": 1, "HAM": 0}}),
    ("looking forward to our collaboration", {"cats": {"SPAM": 0, "HAM": 1}}),
    ("claim your free bonus today", {"cats": {"SPAM": 1, "HAM": 0}}),
    ("your document is ready", {"cats": {"SPAM": 0, "HAM": 1}}),
    ("you have an exclusive invitation", {"cats": {"SPAM": 1, "HAM": 0}}),
    ("hope youre having a great day", {"cats": {"SPAM": 0, "HAM": 1}}),
    ("youve been selected for an amazing offer", {"cats": {"SPAM": 1, "HAM": 0}}),
    ("lets connect soon", {"cats": {"SPAM": 0, "HAM": 1}}),
    ("youve won a free membership", {"cats": {"SPAM": 1, "HAM": 0}}),
    ("your support is important to us", {"cats": {"SPAM": 0, "HAM": 1}}),
]


# Example test data (10 samples)
test_data5 =  [
    ("youve won a free cruise!", {"cats": {"SPAM": 1, "HAM": 0}}),
    ("just wanted to check in on you", {"cats": {"SPAM": 0, "HAM": 1}}),
    ("congratulations! you have a new reward", {"cats": {"SPAM": 1, "HAM": 0}}),
    ("lets schedule a meeting next week", {"cats": {"SPAM": 0, "HAM": 1}}),
    ("youve been selected for a special prize", {"cats": {"SPAM": 1, "HAM": 0}}),
    ("thank you for your feedback", {"cats": {"SPAM": 0, "HAM": 1}}),
    ("win a free smartphone now", {"cats": {"SPAM": 1, "HAM": 0}}),
    ("hope you are doing well", {"cats": {"SPAM": 0, "HAM": 1}}),
    ("urgent: claim your prize today", {"cats": {"SPAM": 1, "HAM": 0}}),
    ("reminder: your appointment is scheduled", {"cats": {"SPAM": 0, "HAM": 1}})
]

nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

# Function to remove stopwords
def remove_stopwords(text):
    words = text.split()
    filtered_words = [word for word in words if word.lower() not in stop_words]
    return ' '.join(filtered_words)

# Clean training data by removing stopwords
train_data5 = [
    (remove_stopwords(text), annotations) for text, annotations in train_data5
]

# Training the model
optimizer5 = nlpTC5.begin_training()
for text5, annotations5 in train_data5:
    doc5 = nlpTC5.make_doc(text5)
    example5 = Example.from_dict(doc5, annotations5)
    nlpTC5.update([example5], drop=0.5, sgd=optimizer5)

# Evaluating the model
y_true5 = []
y_pred5 = []

for text5, annotations5 in test_data5:
    doc5 = nlpTC5(text5)
    y_true5.append(max(annotations5["cats"], key=annotations5["cats"].get))
    y_pred5.append(max(doc5.cats, key=doc5.cats.get))

# Generate classification report
report5 = classification_report(y_true5, y_pred5, target_names=["HAM", "SPAM"])
print("Performance Evaluation Report:\n", report5)

# Function to classify user input emails
def classify_email5(email5):
    processed_email5 = process_text(email5)
    doc5 = nlpTC5(processed_email5)
    spam_score5 = doc5.cats['SPAM']
    ham_score5 = doc5.cats['HAM']
    return "SPAM" if spam_score5 > ham_score5 else "HAM"



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Performance Evaluation Report:
               precision    recall  f1-score   support

         HAM       1.00      0.40      0.57         5
        SPAM       0.62      1.00      0.77         5

    accuracy                           0.70        10
   macro avg       0.81      0.70      0.67        10
weighted avg       0.81      0.70      0.67        10



In [ ]:

# Load a blank model and add text classifier
nlpTC = spacy.blank("en")
textcat = nlpTC.add_pipe("textcat", last=True)

# Add labels for classification
textcat.add_label("SPAM")
textcat.add_label("HAM")

# Expanded example training data
train_data5 = [
    ("You've won a $1000 gift card!", {"cats": {"SPAM": 1, "HAM": 0}}),
    ("Hi, how have you been?", {"cats": {"SPAM": 0, "HAM": 1}}),
    ("Claim your free prize today!", {"cats": {"SPAM": 1, "HAM": 0}}),
    ("Your order has shipped.", {"cats": {"SPAM": 0, "HAM": 1}}),
    ("Congratulations! You are selected for a special offer.", {"cats": {"SPAM": 1, "HAM": 0}}),
    ("Let's meet for lunch soon.", {"cats": {"SPAM": 0, "HAM": 1}}),
    ("Urgent: Update your account information now!", {"cats": {"SPAM": 1, "HAM": 0}}),
    ("Looking forward to our meeting.", {"cats": {"SPAM": 0, "HAM": 1}}),
    ("You have won a free vacation!", {"cats": {"SPAM": 1, "HAM": 0}}),
    ("Just checking in. Hope all is well!", {"cats": {"SPAM": 0, "HAM": 1}}),
    ("Get paid for your opinion!", {"cats": {"SPAM": 1, "HAM": 0}}),
    ("Don't forget about our appointment.", {"cats": {"SPAM": 0, "HAM": 1}}),
    ("You are a lucky winner!", {"cats": {"SPAM": 1, "HAM": 0}}),
    ("Meeting is confirmed for tomorrow.", {"cats": {"SPAM": 0, "HAM": 1}}),
    ("Win a new car! Enter now!", {"cats": {"SPAM": 1, "HAM": 0}}),
    ("Can we reschedule our meeting?", {"cats": {"SPAM": 0, "HAM": 1}}),
    ("You have a package waiting!", {"cats": {"SPAM": 1, "HAM": 0}}),
    ("Your subscription is about to expire.", {"cats": {"SPAM": 0, "HAM": 1}}),
    ("Congratulations! You've been selected!", {"cats": {"SPAM": 1, "HAM": 0}}),
    ("Hope to see you at the event.", {"cats": {"SPAM": 0, "HAM": 1}}),
    ("Earn money from home!", {"cats": {"SPAM": 1, "HAM": 0}}),
    ("Let's catch up soon.", {"cats": {"SPAM": 0, "HAM": 1}}),
    ("You have a new message!", {"cats": {"SPAM": 1, "HAM": 0}}),
    ("Reminder: Your appointment is tomorrow.", {"cats": {"SPAM": 0, "HAM": 1}}),
    ("Claim your free gift now!", {"cats": {"SPAM": 1, "HAM": 0}}),
    ("Looking forward to our lunch.", {"cats": {"SPAM": 0, "HAM": 1}}),
    ("You're a winner!", {"cats": {"SPAM": 1, "HAM": 0}}),
    ("Don't miss this opportunity!", {"cats": {"SPAM": 1, "HAM": 0}}),
    ("Thank you for your purchase!", {"cats": {"SPAM": 0, "HAM": 1}}),
    ("You have won a free trial!", {"cats": {"SPAM": 1, "HAM": 0}}),
    ("It was great seeing you!", {"cats": {"SPAM": 0, "HAM": 1}}),
    ("Exclusive deal just for you!", {"cats": {"SPAM": 1, "HAM": 0}}),
    ("How can I assist you today?", {"cats": {"SPAM": 0, "HAM": 1}}),
    ("You've been selected for a free vacation!", {"cats": {"SPAM": 1, "HAM": 0}}),
    ("Your feedback is important to us.", {"cats": {"SPAM": 0, "HAM": 1}}),
    ("Hurry! Offer expires soon!", {"cats": {"SPAM": 1, "HAM": 0}}),
    ("Let's discuss your project.", {"cats": {"SPAM": 0, "HAM": 1}}),
    ("You've won a $500 shopping spree!", {"cats": {"SPAM": 1, "HAM": 0}}),
    ("How's everything going?", {"cats": {"SPAM": 0, "HAM": 1}}),
    ("Get a free trial of our service!", {"cats": {"SPAM": 1, "HAM": 0}}),
    ("We're excited to have you!", {"cats": {"SPAM": 0, "HAM": 1}}),
    ("Last chance to claim your prize!", {"cats": {"SPAM": 1, "HAM": 0}}),
    ("Thanks for being a valued customer.", {"cats": {"SPAM": 0, "HAM": 1}}),
    ("You've been selected for a special offer!", {"cats": {"SPAM": 1, "HAM": 0}}),
    ("Don't forget to register!", {"cats": {"SPAM": 0, "HAM": 1}}),
    ("Congratulations! You have won a gift card!", {"cats": {"SPAM": 1, "HAM": 0}}),
    ("Your invoice is attached.", {"cats": {"SPAM": 0, "HAM": 1}}),
    ("Get a new laptop for free!", {"cats": {"SPAM": 1, "HAM": 0}}),
    ("Hope to hear from you soon.", {"cats": {"SPAM": 0, "HAM": 1}}),
    ("You've won a luxury cruise!", {"cats": {"SPAM": 1, "HAM": 0}}),
    ("Let's plan a call.", {"cats": {"SPAM": 0, "HAM": 1}}),
    ("You have an important message!", {"cats": {"SPAM": 1, "HAM": 0}}),
    ("Meeting has been postponed.", {"cats": {"SPAM": 0, "HAM": 1}}),
    ("Claim your bonus now!", {"cats": {"SPAM": 1, "HAM": 0}}),
    ("Your application has been approved.", {"cats": {"SPAM": 0, "HAM": 1}}),
    ("Congratulations on your achievement!", {"cats": {"SPAM": 1, "HAM": 0}}),
    ("Let's schedule a follow-up.", {"cats": {"SPAM": 0, "HAM": 1}}),
    ("You are eligible for a discount!", {"cats": {"SPAM": 1, "HAM": 0}}),
    ("Your subscription has been renewed.", {"cats": {"SPAM": 0, "HAM": 1}}),
    ("Win a free iPhone!", {"cats": {"SPAM": 1, "HAM": 0}}),
    ("It was great catching up!", {"cats": {"SPAM": 0, "HAM": 1}}),
    ("You've been chosen for a special reward!", {"cats": {"SPAM": 1, "HAM": 0}}),
    ("We value your opinion.", {"cats": {"SPAM": 0, "HAM": 1}}),
    ("Last chance to win!", {"cats": {"SPAM": 1, "HAM": 0}}),
    ("Don't miss this opportunity to save!", {"cats": {"SPAM": 1, "HAM": 0}}),
    ("Your feedback is appreciated.", {"cats": {"SPAM": 0, "HAM": 1}}),
    ("You have a meeting scheduled.", {"cats": {"SPAM": 0, "HAM": 1}}),
    ("You've won a new TV!", {"cats": {"SPAM": 1, "HAM": 0}}),
    ("Looking forward to our collaboration.", {"cats": {"SPAM": 0, "HAM": 1}}),
    ("Claim your free bonus today!", {"cats": {"SPAM": 1, "HAM": 0}}),
    ("Your document is ready.", {"cats": {"SPAM": 0, "HAM": 1}}),
    ("You have an exclusive invitation!", {"cats": {"SPAM": 1, "HAM": 0}}),
    ("Hope you're having a great day!", {"cats": {"SPAM": 0, "HAM": 1}}),
    ("You've been selected for an amazing offer!", {"cats": {"SPAM": 1, "HAM": 0}}),
    ("Let's connect soon.", {"cats": {"SPAM": 0, "HAM": 1}}),
    ("You've won a free membership!", {"cats": {"SPAM": 1, "HAM": 0}}),
    ("Your support is important to us.", {"cats": {"SPAM": 0, "HAM": 1}}),
    ("You've won a new smartphone!", {"cats": {"SPAM": 1, "HAM": 0}}),
    ("Just a reminder about our meeting tomorrow.", {"cats": {"SPAM": 0, "HAM": 1}}),
    ("Congratulations! You have a new prize waiting.", {"cats": {"SPAM": 1, "HAM": 0}}),
    ("Hope you are doing well!", {"cats": {"SPAM": 0, "HAM": 1}}),
    ("Claim your free vacation!", {"cats": {"SPAM": 1, "HAM": 0}}),
    ("Your feedback is valuable.", {"cats": {"SPAM": 0, "HAM": 1}}),
    ("Win a gift card today!", {"cats": {"SPAM": 1, "HAM": 0}}),
    ("Looking forward to your response.", {"cats": {"SPAM": 0, "HAM": 1}}),
    ("You've been selected for a special event.", {"cats": {"SPAM": 1, "HAM": 0}}),
    ("Thank you for your cooperation.", {"cats": {"SPAM": 0, "HAM": 1}})
]
# Split the data into training and test sets (70-30 split)
train_data5, test_data5 = train_test_split(train_data5, test_size=0.3, random_state=42)

def remove_stopwords(text):
    words = text.split()
    filtered_words = [word for word in words if word.lower() not in stop_words]
    return ' '.join(filtered_words)

# Training the model
optimizer = nlpTC.begin_training()
for text5, annotations in train_data5:  # Use train_data5 here
    text5 = text5.lower()
    text5 = remove_stopwords(text5)
    doc = nlpTC.make_doc(text5)

    example5 = Example.from_dict(doc, annotations)
    nlpTC.update([example5], drop=0.5, sgd=optimizer)  # Use example5 here

# Evaluating the model
y_true5 = []
y_pred5 = []

for text5, annotations in test_data5:  # Use test_data5 here
    doc = nlpTC(text5)
    y_true5.append(max(annotations["cats"], key=annotations["cats"].get))
    y_pred5.append(max(doc.cats, key=doc.cats.get))

# Generate classification report
report5 = classification_report(y_true5, y_pred5, target_names=["HAM", "SPAM"])  # Use y_true5 and y_pred5
print("Performance Evaluation Report:\n", report5)

# Function to classify user input emails
def classify_email(email):
    doc = nlpTC(email)
    spam_score5 = doc.cats['SPAM']
    ham_score5 = doc.cats['HAM']
    return "SPAM" if spam_score5 > ham_score5 else "HAM"


Performance Evaluation Report:
               precision    recall  f1-score   support

         HAM       1.00      0.75      0.86        12
        SPAM       0.83      1.00      0.91        15

    accuracy                           0.89        27
   macro avg       0.92      0.88      0.88        27
weighted avg       0.91      0.89      0.89        27



#Named Entity Recognition

Medium sized dataset in this model is composed of 50 data which also contains text but different to the POS, only named entity are recognized and labeld depending on its role to the given sentence.

##Medium -Low Quality

In [ ]:
# Sample training data (text and true entity annotations)
training_data3= [
    ("Microsoft acquired GitHub for $7.5 billion.", [(0, 9, "ORG"), (18, 24, "ORG")]),
    ("The Eiffel Tower is located in Paris.", [(4, 15, "LOC"), (27, 32, "GPE")]),
    ("Tesla's CEO, Elon Musk, announced new plans.", [(0, 5, "ORG"), (13, 22, "PERSON")]),
    ("Amazon is expanding its services in Germany.", [(0, 6, "ORG"), (30, 37, "GPE")]),
    ("The Great Wall of China attracts millions of tourists.", [(4, 18, "LOC"), (22, 29, "GPE")]),
    ("Netflix released its latest series in July 2023.", [(0, 6, "ORG"), (28, 32, "DATE")]),
    ("Barack Obama served as the 44th President of the United States.", [(0, 12, "PERSON"), (41, 56, "GPE")]),
    ("The Amazon rainforest is vital for our planet.", [(4, 11, "LOC")]),
    ("Apple's new iPhone will be available in October.", [(0, 5, "ORG"), (30, 37, "DATE")]),
    ("Walt Disney World attracts millions of visitors each year.", [(0, 4, "ORG"), (26, 32, "LOC")]),
    ("Facebook is launching a new feature next month.", [(0, 8, "ORG"), (34, 40, "DATE")]),
    ("The Louvre Museum is one of the largest art museums in the world.", [(4, 20, "ORG")]),
    ("Bill Gates founded Microsoft in 1975.", [(0, 10, "PERSON"), (16, 24, "ORG"), (28, 32, "DATE")]),
    ("The Amazon River flows through Brazil and Peru.", [(4, 10, "LOC"), (24, 30, "GPE"), (35, 39, "GPE")]),
    ("Samsung Electronics reported record profits this quarter.", [(0, 18, "ORG")]),
    ("The United Nations was established after World War II.", [(4, 20, "ORG"), (38, 42, "EVENT")]),
    ("Beyoncé performed at the Super Bowl in 2016.", [(0, 7, "PERSON"), (20, 31, "EVENT"), (35, 39, "DATE")]),
    ("The Great Barrier Reef is a UNESCO World Heritage site.", [(4, 22, "LOC")]),
    ("China is hosting the next Olympic Games.", [(0, 5, "GPE"), (27, 34, "EVENT")]),
    ("Stephen Hawking published a new book in 2018.", [(0, 15, "PERSON"), (29, 33, "DATE")]),
    ("Twitter is experiencing an increase in user engagement.", [(0, 6, "ORG")]),
    ("NASA is planning a mission to Mars.", [(0, 4, "ORG"), (25, 29, "LOC")]),
    ("The Tesla Model S is gaining popularity among electric vehicles.", [(4, 15, "ORG")]),
    ("London is the capital city of the United Kingdom.", [(0, 6, "GPE"), (30, 48, "GPE")]),
    ("Harry Potter is a fictional character created by J.K. Rowling.", [(0, 12, "PERSON"), (44, 48, "PERSON")]),
    ("The Eiffel Tower attracts millions of tourists each year.", [(4, 15, "LOC")]),
    ("Elon Musk is the CEO of SpaceX.", [(0, 9, "PERSON"), (24, 30, "ORG")]),
    ("The Pacific Ocean is the largest ocean on Earth.", [(4, 17, "LOC")]),
    ("Sony unveiled a new gaming console this year.", [(0, 4, "ORG"), (29, 33, "DATE")]),
    ("Barack Obama delivered a speech at the Democratic National Convention.", [(0, 12, "PERSON"), (37, 47, "EVENT")]),
    ("Google launched a new AI tool last week.", [(0, 6, "ORG"), (30, 34, "DATE")]),
    ("The Statue of Liberty is in New York City.", [(4, 21, "LOC"), (25, 38, "GPE")]),
    ("Oprah Winfrey will host a new talk show.", [(0, 12, "PERSON")]),
    ("The Great Pyramid of Giza is a marvel of engineering.", [(4, 30, "LOC")]),
    ("Elon Musk plans to send humans to Mars.", [(0, 9, "PERSON"), (29, 33, "LOC")]),
    ("The Academy Awards are held annually.", [(4, 20, "EVENT")]),
    ("Nokia announced a partnership with Microsoft.", [(0, 5, "ORG"), (36, 44, "ORG")]),
    ("Queen Elizabeth II reigned for decades.", [(0, 15, "PERSON")]),
    ("The Amazon is the largest rainforest in the world.", [(4, 11, "LOC")]),
    ("Facebook acquired Instagram in 2012.", [(0, 8, "ORG"), (21, 29, "ORG"), (33, 37, "DATE")])
]

# Initialize lists for storing true and predicted entities
all_true_entities3 = []
all_pred_entities3 = []

# Iterate through training data
for text, true_entities in training_data3:
    # Run NER model
    doc = nlp(text)

    # Predicted entities from the model
    pred_entities3 = [(ent.start_char, ent.end_char, ent.label_) for ent in doc.ents]

    # Store true and predicted entities for all examples
    all_true_entities3.extend([(ent[0], ent[1], ent[2]) for ent in true_entities])
    all_pred_entities3.extend(pred_entities3)

# Convert to binary classification (1 for correct entity, 0 for incorrect)
y_true3 = [1 if ent in all_true_entities3 else 0 for ent in all_pred_entities3]
y_pred3 = [1 for _ in all_pred_entities3]  # Assuming all predictions are correct for now

# Calculate Precision, Recall, F1, and Accuracy
precision3 = precision_score(y_true3, y_pred3)
recall3 = recall_score(y_true3, y_pred3)
f13 = f1_score(y_true3, y_pred3)
accuracy3 = accuracy_score(y_true3, y_pred3)

print("Accuracy:", accuracy3)
print("Precision:", precision3)
print("Recall:", recall3)
print("F1-Score:", f13)


Accuracy: 0.28205128205128205
Precision: 0.28205128205128205
Recall: 1.0
F1-Score: 0.44


#Summarizer

In [2]:
import spacy
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from nltk.tokenize import sent_tokenize
import nltk
nltk.download('punkt')

# Load spaCy model (small for demonstration purposes)
nlp = spacy.load("en_core_web_sm")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


The data used in this model is an article that was gathered and collected from GMA NEWS online, the text article is then fed to the previously created summarizer in which becomes the reference summary that will be used to identify the performance of the summarizer.

##Small - Low Quality

In [3]:
# Original text to summarize
text = """CEBU CITY, Philippines – Former Cebu City Mayor Tomas Osmeña vowed to reveal more information about the extra-judicial killings (EJKs) here in the next House committee hearing. Osmeña made this statement on Thursday, September 26, after announcing that the hearing scheduled this Friday, September 27, has been postponed to mid-October. However, the former mayor, who was once again invited to shed more light about reports of EJKs here, shared that this development would allow him to present and reveal more information, including records from victims of EJKs. “I’m going to use the extra time to incorporate additional information into my presentation. I already have plenty of information, both from my own records and from victims who have approached me in private,” he wrote. “Plenty of people have come forward. If anyone else wants to talk to me privately, as always I can be contacted directly at 0917 329 9999,” added Osmeña. Osmeña served as a resource speaker in the ongoing House quad committee hearing on EJKs and Philippine Offshore Gambling Operators (Pogos)."""

# Reference summary with exact sentences from the original text
reference_summary = """CEBU CITY, Philippines – Former Cebu City Mayor Tomas Osmeña  to reveal more information about the extra-judicial killings (EJKs) here in the next House committee hearing. However, the former mayor, who was once again invited to shed more light about reports of EJKs here, shared that this development would allow him to present and reveal more information, including records from victims of EJKs. Osmeña served as a resource speaker in the ongoing House quad committee hearing on EJKs and Philippine Offshore Gambling Operators (Pogos).
"""

# Extractive summarization function
def extractive_summary(text, num_sentences=3):

    doc = nlp(text)
    sentences = [sent.text.strip() for sent in doc.sents]
    return sentences[:num_sentences]  # Select the first 'num_sentences' sentences

# Generate summary using the extractive method
generated_summary = extractive_summary(text)
generated_summary_text = ' '.join(generated_summary)

# Tokenize the reference summary into sentences
reference_sentences = sent_tokenize(reference_summary)

# Tokenize the original text into sentences
original_sentences = sent_tokenize(text)

# Create binary relevance labels for each sentence in the original text
# y_true: 1 if the sentence is in the reference summary, else 0
# y_pred: 1 if the sentence is in the generated summary, else 0
y_true = [1 if sent in reference_sentences else 0 for sent in original_sentences]
y_pred = [1 if sent in generated_summary else 0 for sent in original_sentences]

# Calculate Precision, Recall, F1 Score, and Accuracy
precision = precision_score(y_true, y_pred)
recall = recall_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred)
accuracy = accuracy_score(y_true, y_pred)

# Output results
print(f"Generated Summary: {generated_summary_text}")
print(f"Accuracy: {accuracy:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score: {f1:.2f}")
print(generated_summary_text)

Generated Summary: CEBU CITY, Philippines – Former Cebu City Mayor Tomas Osmeña vowed to reveal more information about the extra-judicial killings (EJKs) here in the next House committee hearing. Osmeña made this statement on Thursday, September 26, after announcing that the hearing scheduled this Friday, September 27, has been postponed to mid-October. However, the former mayor, who was once again invited to shed more light about reports of EJKs here, shared that this development would allow him to present and reveal more information, including records from victims of EJKs.
Accuracy: 0.62
Precision: 0.33
Recall: 0.50
F1 Score: 0.40
CEBU CITY, Philippines – Former Cebu City Mayor Tomas Osmeña vowed to reveal more information about the extra-judicial killings (EJKs) here in the next House committee hearing. Osmeña made this statement on Thursday, September 26, after announcing that the hearing scheduled this Friday, September 27, has been postponed to mid-October. However, the former ma